<a href="https://colab.research.google.com/github/nak650228/ITEC/blob/20211010/VideoRestore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#◢ Video Restoration Prework

プリワークでは、Youtubeまたはファイル指定により動画ファイルをダウンロードし後後、DeepLearningによる補正を行う前処理を行います。

その他にも以下の修正を施します。

　　・DeOldifyによるモノクロ画像のカラー化

　　・Deep Remasterによるノイズ等の除去

　　・アンシャープマスクの適用（OpenCVによる）

　　・Microsoft Bringing-Old-Photos-Back-to-LifeまたはGFPGANによる顔画像の修復  


今後の機能追加

　　・超解像度化（4Kもしくは2Kに）

　 　　 ToDo TecoGANもしくはiseebetterを適用する（通常動画）

　　　　　　RealESRGANまたは Waifuを適用する（アニメなど）

　　・シーンの分割（トランジションを判断して、複数の動画ファイルに分割します。  

　　   　ToDo  分割したシーンごとに変換を行うよう、各処理を関数化する

　　・DAINを使ったフレーム補完

　　・音声データの取得と変換後ビデオへの追加  

　　   　ToDo  選択した動画領域から、音声データを取得、最終的結果にアペンドさせる


  
各処理ごとの出力結果が、ユーザのGoogle Drive上のMovieフォルダに作られます。

#◢ 初期設定

In [5]:
#@title ライブラリ等のインストール
%cd /content
!pip install youtube_dl
!pip install ffmpeg
!pip install ffmpeg-python
#!pip install torchvision==0.5
!pip install torchvision
#!pip install torch==1.4
!pip install torch==1.9.0
#!pip install scipy==1.2.0
!pip install scipy
#!pip install imgaug==0.2.5
!pip install imgaug
#!pip install tensorflow==1.15.5
!pip install tensorflow

#シーン分割
!pip install scenedetect[opencv,progress_bar]

!pip install subprocess

import subprocess
import tensorflow as tf
import youtube_dl
import ffmpeg
import numpy as np
import imageio
import cv2
import torch
import glob
import shutil
import moviepy.editor as mpy
import os
from IPython.display import clear_output
torch.backends.cudnn.benchmark=True

clear_output()


In [ ]:
#@title Microsoft Bringing-Old-Photos-Back-to-Lifeのリポジトリーをクローンする
%cd /content
!git clone https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life.git photo_restoration

#@markdown Microsoft Bringing Old-Photos-Back-to-Lifeの学習済みモデルをダウンロード
# pull the syncBN repo
%cd photo_restoration/Face_Enhancement/models/networks
!git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
!cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
%cd ../../../

%cd Global/detection_models
!git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
!cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
%cd ../../

# download the landmark detection model
%cd Face_Detection/
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2
%cd ../

# download the pretrained model
%cd Face_Enhancement/
!wget https://facevc.blob.core.windows.net/zhanbo/old_photo/pretrain/Face_Enhancement/checkpoints.zip
!unzip -o checkpoints.zip
%cd ../

%cd Global/
!wget https://facevc.blob.core.windows.net/zhanbo/old_photo/pretrain/Global/checkpoints.zip
!unzip -o checkpoints.zip
%cd ../

! pip install -r requirements.txt

clear_output()

In [ ]:
#@title DeOldifyの初期設定
%cd /content

!git clone https://github.com/jantic/DeOldify.git DeOldify
%cd DeOldify

#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')

from os import path

!pip install -r colab_requirements.txt

import fastai
from deoldify.visualize import *
from pathlib import Path
torch.backends.cudnn.benchmark=True
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

!mkdir 'models'
!wget https://data.deepai.org/deoldify/ColorizeVideo_gen.pth -O ./models/ColorizeVideo_gen.pth

colorizer = get_video_colorizer()

clear_output()

In [ ]:
#@title ##**GFPGANをGithubからクローン**

# Clone GFPGAN and enter the GFPGAN folder
%cd /content
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

# Set up the environment
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr
# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions
# Download the pre-trained model
!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models
clear_output()

In [ ]:
#@title ##**Clone the repository of DeepRemaster** { display-mode: "form" }
%cd /content
!git clone https://github.com/satoshiiizuka/siggraphasia2019_remastering.git DeepRemaster
!cp -r /content/video.mp4 /content/DeepRemaster/
%cd /content/DeepRemaster

!wget --continue -O model/remasternet.pth.tar -- http://iizuka.cs.tsukuba.ac.jp/data/remasternet.pth.tar
clear_output()

#◢ 関数定義

In [ ]:
#@title 各種関数の定義
#@markdown **decomposit_video(展開先フォルダ, ソースビデオファイル)  　　　　ビデオを静止画フレームに分解する**
#@markdown **unsharp_mask(ソースイメージ, 先鋭化の強さ)　　　　　　　　　　静止画のエッジなどを強調する**

def decomposit_video(video_folder,video_file):
  if os.path.isdir(video_folder):
    shutil.rmtree(video_folder)

  os.mkdir(video_folder)

  os.chdir(video_folder)

  decompose_command = 'ffmpeg -i ' + video_file + ' %09d.png'
  subprocess.run(decompose_command, shell=True)
  #!ffmpeg -i /content/video.mp4 %09d.png

  clear_output()

def composit_video(video_folder, video_file, frame_rate):
  if os.path.isfile(video_file):
    os.remove(video_file)

  compose_command='ffmpeg -f image2 -framerate ' + str(frame_rate) + ' -i ' + video_folder + '/%09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p ' + video_file
  subprocess.run(compose_command, shell=True)
  clear_output()

def UnSharpMask(image_file, k):
    kernel = np.array([[-k/9.0, -k/9.0, -k/9.0],
                    [-k/9.0, 1 + (8 * k)/9.0, -k/9.0],
                    [-k/9.0, -k/9.0, -k/9.0]])
    dst = cv2.filter2D(image_file, -1, kernel)
    return dst

def make_sharp_kernel(k: int):
  return np.array([
    [-k / 9, -k / 9, -k / 9],
    [-k / 9, 1 + 8 * k / 9, k / 9],
    [-k / 9, -k / 9, -k / 9]
  ], np.float32)

def unsharp_mask_movie(video_folder, video_file, k, frame_rate):
  decomposit_video(video_folder,video_file)

  files = glob.glob(video_folder+'/*.png')
  for ifile in files:
    print("Now processing ",ifile)
    imageblur = cv2.imread(ifile, 0)
#    imagesharp = UnSharpMask(imageblur,k)

    kernel=make_sharp_kernel(1.0)
    imagesharp=cv2.filter2D(imageblur,-1,kernel).astype("uint8")
    cv2.imwrite(ifile, imagesharp)

  composit_video(video_folder, video_file, frame_rate)


from PIL import Image
import numpy as np
import torch
import torch.nn.functional as F
import torchvision

def load_tensor(image_file):
    with Image.open(image_file) as img:
        array = np.asarray(img, np.float32).transpose([2, 0, 1]) / 255.0
        tensor = torch.as_tensor(np.expand_dims(array, axis=0))  # rank 4
    return tensor


def sharpen_filter(image_file):
    kernel = np.array([[-2, -2, -2], [-2, 32, -2], [-2, -2, -2]], np.float32) / 16.0  # convolution filter
    with torch.no_grad():
        # [out_ch, in_ch, .., ..] : channel wiseに計算
        sharpen_k = torch.as_tensor(kernel.reshape(1, 1, 3, 3))

        color = load_tensor(image_file)  # color image [1, 3, H, W]
        # channel-wise conv(大事)　3x3 convなのでPadding=1を入れる
        multiband = [F.conv2d(color[:, i:i + 1,:,:], sharpen_k, padding=1) for i in range(3)]
        sharpened_image = torch.cat(multiband, dim=1)
        torchvision.utils.save_image(sharpened_image, image_file)


def sharpen_movie(video_folder, video_file, k, frame_rate):
  decomposit_video(video_folder,video_file)

  files = glob.glob(video_folder+'/*.png')
  for ifile in files:
    print("Now processing ",ifile)
    sharpen_filter(ifile)

  composit_video(video_folder, video_file, frame_rate)


def colorize_video(source_video, output_video, render_factor):
  print(source_video)
  print(output_video)

  if os.path.isfile("/content/DeOldify/video"):
    shutil.rmtree("/content/DeOldify/video")

  !mkdir -p '/content/DeOldify/video/source'

  command1="cp -r " + source_video + " /content/DeOldify/video/source/video.mp4"
  subprocess.run(command1,shell=True)
  print(command1)
#  !cp -r str(source_video) /content/DeOldify/video/source/video.mp4
  video_path = colorizer.colorize_from_file_name('/content/DeOldify/video/source/video.mp4', render_factor)
 
  command2="cp -r /content/DeOldify/video/result/video.mp4 " + output_video
  subprocess.run(command2,shell=True)
#  !cp -r /content/DeOldify/video/result/video.mp4 str(output_video)
  print(command2)
  if os.path.isfile("/content/DeOldify/video/result/video.mp4"):
    os.remove("/content/DeOldify/video/result/video.mp4")

#◢ 画像のダウンロード

In [ ]:
#@title 割り当てられたGPUの確認
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# 16GB: Can handle 720p. 1080p will procude an out-of-memory error. 
# 8GB: Can handle 480p. 720p will produce an out-of-memory error.

!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla P100-PCIE-16GB, 460.32.03, 16280 MiB


In [ ]:
#@title **Googleドライブの追加**
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print('Google Drive connected.')

Mounted at /content/drive
Google Drive connected.


In [7]:
#@title ##**ビデオのダウンロード** { display-mode: "form" }
#@markdown *ビデオへのリンク（YouTubeやTwitterなど）を入力するか、source_urlフィールドを空白にしてください（空白にした場合、コンピューターからビデオをアップロードするよう求められます）。*
#@markdown *プロジェクト名は任意でOKです。一応最後にGoogle Driveにプロジェクト名で指定したディレクトリが作られて、途中経過も含めて全ての動画ファイルがコピーされます*

import youtube_dl
import cv2
from IPython.display import clear_output

%cd /content

projectname = 'Miyazaki_Kiyoshiro' #@param {type:"string"}

source_url = 'https://www.youtube.com/watch?v=IiAQpmP4W9Q' #@param {type:"string"}

%cd /content
! rm -f /content/*.mp4

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")
  file_name = "downloaded_video." + fn.split(".")[-1]
  !mv -f $fn $file_name

else:
  try:
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': 'downloaded_video.mp4',
        }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([source_url])
    file_name = 'downloaded_video.mp4'
  
  except BaseException:
    !wget $source_url
    fn = source_url.split('/')[-1]
    os.rename(fn, fn.replace(" ", ""))
    fn = fn.replace(" ", "")
    file_name = "downloaded_video." + fn.split(".")[-1]
    !mv -f $fn $file_name

!cp -r /content/downloaded_video.mp4 /content/video.mp4

clear_output()

fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))
width_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_WIDTH))
height_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_HEIGHT))
print ("FPS of VIDEO: ",fps_of_video)
print ("Frames of VIDEO: ",frames_of_video)
print ("Width of VIDEO: ",width_of_video)
print ("Height of VIDEO: ",height_of_video)



FPS of VIDEO:  15
Frames of VIDEO:  936
Width of VIDEO:  480
Height of VIDEO:  360


In [ ]:
#@title ##**ダウンロードした動画を表示** { display-mode: "form" }
#@markdown *ビデオは横640ドットに拡大/縮小されて表示されます*
import moviepy.editor as mpy

what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/downloaded_video.mp4", autoplay=1, maxduration=6000,width=640))
else:
  files.download('/content/downloaded_video.mp4')

In [ ]:
#@title ##**動画の調整** { display-mode: "form" }

#@markdown *1分以上の動画をダウンロードすることはお勧めできません。また、タイトルに「スペース」や「ドット」が含まれている動画はアップロードしないでください。*
#@markdown *実行中にエラーが発生した場合は、このブロックを再度実行します。*
#@markdown *動画の長さを変更することができます。*

#@markdown **動画の長さを変更する場合は、その開始時間と終了時間を指定して下さい。**
target_start = '00:00:27' #@param {type:"string"}
target_end = '00:00:42' #@param {type:"string"}


if os.path.isfile("/content/cropped_video.mp4"):
    os.remove("/content/cropped_video.mp4")

!ffmpeg -i /content/downloaded_video.mp4  -ss $target_start -to $target_end /content/cropped_video.mp4

if os.path.isfile("/content/video.mp4"):
    os.remove("/content/video.mp4")

!cp /content/cropped_video.mp4 /content/video.mp4

#@markdown **モノクロ動画にAIで色を付ける場合は有効にしてください。**
is_deoldify = True #@param {type:"boolean"}


#@markdown **Deoldifyのレンダリングファクターを指定します。(お勧めは10～25）**
render_factor = 13  #@param {type: "slider", min: 5, max: 44}

#@markdown **アンシャープマスクを適用する場合は有効にして下さい。**
is_unsharp = True #@param {type:"boolean"}


#@markdown **DeepRemasterを有効にすると、低画質の動画からノイズなどを除去できます。**
is_DeepRemaster = True #@param {type:"boolean"}

#@markdown **GANを使って、動画の顔部分を書き換えます。大体はがっかりしますが、時々きれいに修正されることがあります**

#@markdown **Microsoft Bringing-Old-Photos-Back-to-LifeまたはGFPGANを選択して、いずれかの手法で人物の精彩化を試みます。**
which_FaceGAN = 'Microsoft' #@param ["None", "GFPGAN", "Microsoft"] {allow-input: true}

clear_output()

In [9]:
#@title ##**サイズ調整後の動画を表示（修復対象）** { display-mode: "form" }

what_next = 'play' #@param ["play", "download"]
if what_next == "play":
#  display(mpy.ipython_display("/content/video.mp4", height=400, autoplay=1, maxduration=600))
  display(mpy.ipython_display("/content/video.mp4", autoplay=1, maxduration=600, width=640))
else:
  files.download('/content/video.mp4')

In [2]:
%cd /content
!git clone https://github.com/JoeyBallentine/Video-Inference.git

/content
Cloning into 'Video-Inference'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 131 (delta 53), reused 78 (delta 5), pack-reused 0
Receiving objects: 100% (131/131), 20.40 MiB | 38.32 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [3]:
%cd /content/Video-Inference/models/
!wget https://drive.google.com/file/d/13FPxKE6q7tuRrfhTE7GB040jBeURBj58/view
!wget https://drive.google.com/file/d/1ie1F7wJcO4mhNWK8nPX7F0LgOoPzCwEu/view
!wget https://u.pcloud.link/publink/show?code=XZeXKLXZdqXM0uCIGvH7IFyg0sSwC7dl2y2X


/content/Video-Inference/models
--2021-10-18 14:51:42--  https://drive.google.com/file/d/13FPxKE6q7tuRrfhTE7GB040jBeURBj58/view
Resolving drive.google.com (drive.google.com)... 173.194.218.101, 173.194.218.139, 173.194.218.138, ...
Connecting to drive.google.com (drive.google.com)|173.194.218.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view’

view                    [ <=>                ]  64.24K  --.-KB/s    in 0.003s  

2021-10-18 14:51:42 (18.0 MB/s) - ‘view’ saved [65783]

--2021-10-18 14:51:42--  https://drive.google.com/file/d/1ie1F7wJcO4mhNWK8nPX7F0LgOoPzCwEu/view
Resolving drive.google.com (drive.google.com)... 173.194.218.100, 173.194.218.101, 173.194.218.139, ...
Connecting to drive.google.com (drive.google.com)|173.194.218.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view.1’

view.1                  [ <=>                ]  64.24K  --.-KB/

In [10]:
%cd /content/Video-Inference/
!python run.py ./models/TecoGAN_BI_iter500000.pth --input "./input/video.mp4" --output "./output/output_video.mp4"

/content/Video-Inference
N/A% (0 of 936) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100% (936 of 936) |#######################| Elapsed Time: 0:03:27 Time:  0:03:27


In [ ]:
#@title  ##**動画にアンシャープマスクをかける**
if is_unsharp == True:
  sharpen_movie("/content/datas", "/content/video.mp4", 1.0, fps_of_video)

In [ ]:
#@title ##**動画をシーンごとに分割する** { display-mode: "form" }

splitted_folder="/content/movies"
source_video="/content/video.mp4"
if os.path.isdir(splitted_folder):
    shutil.rmtree(splitted_folder)
os.mkdir(splitted_folder)

#シーンごとに動画を分割する
split_command="scenedetect -i " + source_video + " -o " + splitted_folder + " detect-content -t 27 list-scenes save-images split-video"
subprocess.run(split_command,shell=True)


CompletedProcess(args='scenedetect -i /content/video.mp4 -o /content/movies detect-content -t 27 list-scenes save-images split-video', returncode=0)

# #◢ モノクロ動画に色を付ける

In [ ]:
#@title Deoldifyによるモノクロ動画のカラー化
if is_deoldify == True:

  %cd /content/DeOldify

  if os.path.isfile("/content/DeOldify/video"):
    shutil.rmtree("/content/DeOldify/video")

  !mkdir -p '/content/DeOldify/video/source'

  !cp -r /content/video.mp4 /content/DeOldify/video/source/video.mp4
  video_path = colorizer.colorize_from_file_name('/content/DeOldify/video/source/video.mp4', render_factor)
  !cp -r /content/DeOldify/video/result/video.mp4 /content/colorized_video.mp4
  !cp -r /content/colorized_video.mp4 /content/video.mp4
  if os.path.isfile("/content/DeOldify/video/result/video.mp4"):
    os.remove("/content/DeOldify/video/result/video.mp4")


#colorize_video('/content/video.mp4', '/content/colorized_video.mp4', render_factor)
#!cp /content/colorized_video /content/video.mp4

/content/DeOldify


Video created here: video/result/video.mp4


# #◢ ノイズ除去(Deep Remaster)

In [ ]:
#@title ##**Remove frame noise** { display-mode: "form" }
%cd /content/DeepRemaster
command = "python remaster.py --input /content/video.mp4 --disable_colorization --gpu --mindim " + str(height_of_video)

subprocess.run(command,shell=True)
#!python remaster.py --input /content/video.mp4 --disable_colorization --gpu --mindim 1080
#!python remaster.py --input /content/video.mp4 --disable_colorization --gpu
#clear_output()

#◢ Microsoft Bringing-Old-Photos-Back-to-Lifeによる画像修正

In [ ]:
#@title 動画ファイルを画像ファイルに分解
# ffmpeg extract - Generating individual frame PNGs from the source file.

if which_FaceGAN == "Microsoft":

  %cd /content/photo_restoration

  FRAME_INPUT_DIR = "/content/photo_restoration/input_frames"
  FRAME_OUTPUT_DIR = "/content/photo_restoration/output_frames"
  INPUT_FILEPATH = "/content/video.mp4"

  if os.path.isfile(FRAME_INPUT_DIR):
    shutil.retree(FRAME_INPUT_DIR)

  %shell mkdir -p '{FRAME_INPUT_DIR}'

  %shell ffmpeg -i '{INPUT_FILEPATH}' '{FRAME_INPUT_DIR}/%05d.png'

  if os.path.isfile(FRAME_OUTPUT_DIR):
    shutil.retree(FRAME_OUTPUT_DIR)
  
  %shell mkdir -p '{FRAME_OUTPUT_DIR}'


  png_generated_count_command_result = %shell ls '{FRAME_INPUT_DIR}' | wc -l
  from IPython.display import clear_output

  pngs_generated_count = int(png_generated_count_command_result.output.strip())


  #print(f"Input FPS: {fps}")
  print(f"{pngs_generated_count} frame PNGs generated.")

  # Checking if PNG do have alpha
  import subprocess as sp
  %cd {FRAME_INPUT_DIR}
  channels = sp.getoutput('identify -format %[channels] 00001.png')
  print (f"{channels} detected")

  # Removing alpha if detected
  if "a" in channels:
    print("Alpha detected and will be removed.")
    print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))

  %cd /content/photo_restoration
  input_folder = FRAME_INPUT_DIR
  output_folder = FRAME_OUTPUT_DIR

  !rm -rf /content/photo_restoration/output_frames/*

  print (input_folder)
  print (output_folder)

  import os
  basepath = os.getcwd()
  #input_path = os.path.join(basepath, input_folder)
  #output_path = os.path.join(basepath, output_folder)
  #os.mkdir(output_path)
  #!rm -rf output_folder
  #os.mkdir(output_folder)

  !python run.py --input_folder /content/photo_restoration/input_frames --output_folder /content/photo_restoration/output_frames --GPU 0

#create video

  %cd /content/photo_restoration/output_frames/final_output
  !ffmpeg  -pattern_type glob -i '*.png' -c:v h264_nvenc -pix_fmt yuv420p /content/beautified_video.mp4
  !rm -f /content/video.mp4
  !cp /content/beautified_video.mp4 /content/video.mp4
  clear_output()

#◢ GFPGANによる顔画像の修復

In [ ]:
#@title ##**GFPGANを使って動画ファイル中の顔画像を修復** { display-mode: "form" }

if which_FaceGAN == "GFPGAN":

  upload_folder = "/content/GFPGAN/inputs/upload"
  if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)

  os.mkdir(upload_folder)

  %cd /content/GFPGAN/inputs/upload

  !ffmpeg -i /content/video.mp4 %09d.png

# Now we use the GFPGAN to restore the above low-quality images
# We use [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN) for enhancing the background (non-face) regions

  %cd /content/GFPGAN
  !rm -rf results
  !python inference_gfpgan.py --upscale 2 --test_path inputs/upload --save_root results --model_path experiments/pretrained_models/GFPGANCleanv1-NoCE-C2.pth --bg_upsampler realesrgan

  if os.path.isfile("/content/restored_video.mp4") :
    !rm -f /content/restored_video.mp4

  !ffmpeg -f image2 -framerate 30 -i /content/GFPGAN/results/restored_imgs/%09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/beautified_video.mp4
  !rm -f /content/video.mp4
  !cp /content/beautified_video.mp4 /content/video.mp4

  clear_output()

#◢ 最終処理

In [ ]:
#@title ファイルのバックアップ

ProjectDir="/content/drive/MyDrive/Movie/"+str(projectname)
print("ProjectDir")

if os.path.isfile(ProjectDir):
    shutil.rmtree(ProjectDir)

os.mkdir(ProjectDir)
os.chdir(ProjectDir)
!mv /content/*.mp4 .

ProjectDir
